In [1]:
# This file was *autogenerated* from the file UTnUtil.sage
from sage.all_cmdline import *   # import sage library

_sage_const_1 = Integer(1); _sage_const_2 = Integer(2); _sage_const_0 = Integer(0)############################# Functions ######################################

#Gets the unipotent upertriangular nxn matrices over the field f
def getUTn(n: int, f: sage.rings.finite_rings.finite_field_prime_modn):
    toRet = []
    fillings = getListOfLength(((n-1)*n)/2, f)

    for filling in fillings:
        toRet.append(getUTnMatFromList(n, f, filling))
    return toRet

In [2]:
def getElemUTn(n: int, f: sage.rings.finite_rings.finite_field_prime_modn):
    toRet = []
    fillings = []
    l = ((n-1)*n)/2
    for i in range(l):
        toAdd = [0]*int(l)
        toAdd[i] = 1
        fillings.append(toAdd)

    for filling in fillings:
        toRet.append(getUTnMatFromList(n, f, filling))
    return toRet

In [3]:
def getUTnMatFromList(n: int, f: sage.rings.finite_rings.finite_field_prime_modn, l: list):
    #((n-1)*n)/2 is the number of entries above the diagonal
    if len(l) != ((n-1)*n)/2:
        raise NameError('Invalid array length')
    
    mat = copy(MatrixSpace(f,n,n).identity_matrix())
    current = 0
    for i in range(n):
        for j in range(n-i-1):
            mat[i,j+i+1] = l[current]
            current += 1
    return mat

In [4]:
#Recursivley generates all lists of length n composed of element of f
#Returns a list of all such lists
def getListOfLength(n: int, f: sage.rings.finite_rings.finite_field_prime_modn):
    toRet = []
    if n == 1:
        for elem in f:
            toRet.append([elem])
        return toRet
    
    for elem in f:
        beginnings = getListOfLength(n-1,f)
        for beginning in beginnings:
            beginning.append(elem)
            toRet.append(beginning)
    return toRet

In [5]:
#Remember, python uses 0 indexed lists, so we have to offset our entries by 1
def blackout(mat: sage.matrix, entries: list):
    for entry in entries:
        mat[entry[0],entry[1]] = 0
    return mat

def getConjClasses(toSearch, toBlackout, n,f, v=False):
    found = []
    classes = []
    conjugators = getElemUTn(n,f)
    for mat in toSearch:
        if mat not in found:
            if v:
                #print("Found new class with rep:")
                #print(mat)
                print("Current number of classes: " + str(len(classes)+1)) #have to add one since current isn' added yet
            stack = [mat]
            found.append(mat)
            conjClass = [mat]
            while len(stack) > 0:
                n = stack.pop()
                for c in conjugators:
                    raw = c*n*c.inverse()
                    current = blackout(raw,toBlackout)
                    if current not in conjClass:
                        found.append(current)
                        conjClass.append(current)
                        stack.append(current)
            classes.append(conjClass)
            if v:
                print("This conjugacy class has size: " + str(len(conjClass)))
    return classes

In [6]:
def blackoutForBottomLeft(mat: sage.matrix, n: int):
    toBlackout = []
    for i in range(n):
        for j in range(n):
            if j >= i:
                toBlackout.append([i,j])
    return blackout(mat, toBlackout)

In [7]:
def fillMatFromList(toFill: list, entries: list, n: int, f: sage.rings.finite_rings.finite_field_prime_modn):
    if len(toFill) != len(entries):
        raise NameError('Inputs must have the same length')

    mat = MatrixSpace(f,n,n).identity_matrix() - MatrixSpace(f,n,n).identity_matrix()
    for i in range(len(entries)):
        e = toFill[i]
        mat[e[0],e[1]] = entries[i]

    return mat

In [8]:
def getMatsOfShape(shape: list, n: int, f: sage.rings.finite_rings.finite_field_prime_modn):
    toRet = []
    fillings = getListOfLength(len(shape),f)
    for filling in fillings:
        toRet.append(fillMatFromList(shape, filling, n, f))
    return toRet

In [9]:
def printClasses(classes):
    for c in classes:
        print("Size: " + str(len(c)))
        for e in c:
            print(e)
            print("------")
        print("######")
    print("Number of classes: " + str(len(classes)))

In [14]:
p = 2
f = GF(p)
n = 7
M = MatrixSpace(f,n,n)

def getHook(n,h):
    if n-h < 1:
        raise NameError('Height is too large for matrix size')

    toRet = []
    for i in range(1,n):
        toRet.append([0,i])
    for i in range(1,h):
        toRet.append([i,n-1])

    return toRet

print("Running")

Running


In [15]:
def getBoxesAbovePath(path, n):
    rows = {}
    for b in path:
        if b[0] not in rows.keys():
            rows[b[0]] = b[1]
        if rows[b[0]] < b[1]:
            rows[b[0]] = b[1]

    toRet = []
    for k in rows.keys():
        for i in range(rows[k]+1,n):
            toRet.append([k,i])
    return toRet

In [16]:
# 4x4 hook in a 5x5 matrix path = [[0,1],[0,2],[0,3],[0,4],[1,4],[2,4],[3,4]]
# 5x5 hook in a 6x6 matrix path = [[0,1],[0,2],[0,3],[0,4],[0,5],[1,5],[2,5],[3,5],[4,5]]
# 6x6 hook in a 7x7 matrix path = [[0,1],[0,2],[0,3],[0,4],[0,5],[0,6],[1,6],[2,6],[3,6],[4,6],[5,6]]
#2 3x3 hooks in a 6x6 matrix path = [[0,1],[0,2],[0,3],[1,3],[2,3],[2,4],[2,5],[3,5],[4,5]]
#1 3x3 hook and 1 4x4 hook in a 7x7 matrix path = [[0,1],[0,2],[0,3],[1,3],[2,3],[2,4],[2,5],[2,6],[3,6],[4,6],[5,6]]
#2 4x4 hooks in a 8x8 matrix path = [[0,1],[0,2],[0,3],[0,4],[1,4],[2,4],[3,4],[3,5],[3,6],[3,7],[4,7],[5,7],[6,7]]
#3 3x3 hooks in a 8x8 matrix path = [[0,1],[0,2],[0,3],[1,3],[2,3],[2,4],[2,5],[3,5],[4,5],[4,6],[4,7],[5,7],[6,7]]
#3x3,2x2,3x3 path = [[0,1],[0,2],[0,3],[1,3],[2,3],[2,4],[3,4],[3,5],[3,6],[4,6],[5,6]]
#3x3,3x3,4x4 in a 9x9 path = [[0,1],[0,2],[0,3],[1,3],[2,3],[2,4],[2,5],[3,5],[4,5],[4,6],[4,7],[4,8],[5,8],[6,8],[7,8]]
#3x3,4x4,3x3 in a 9x9 path = [[0,1],[0,2],[0,3],[1,3],[2,3],[2,4],[2,5],[2,6],[3,6],[4,6],[5,6],[5,7],[5,8],[6,8],[7,8]]

blackoutboxes = getBoxesAbovePath(path, n)
print(blackoutboxes)
print("Getting mats")
mats = getMatsOfShape(path, n, f)
print("Getting classes")

classes = getConjClasses(mats,blackoutboxes,n,f,True)

[[0, 4], [0, 5], [0, 6], [1, 4], [1, 5], [1, 6], [2, 5], [2, 6]]
Getting mats
Getting classes
Current number of classes: 1
This conjugacy class has size: 1
Current number of classes: 2
This conjugacy class has size: 4
Current number of classes: 3
This conjugacy class has size: 2
Current number of classes: 4
This conjugacy class has size: 1
Current number of classes: 5
This conjugacy class has size: 2
Current number of classes: 6
This conjugacy class has size: 4
Current number of classes: 7
This conjugacy class has size: 2
Current number of classes: 8
This conjugacy class has size: 8
Current number of classes: 9
This conjugacy class has size: 8
Current number of classes: 10
This conjugacy class has size: 8
Current number of classes: 11
This conjugacy class has size: 8
Current number of classes: 12
This conjugacy class has size: 1
Current number of classes: 13
This conjugacy class has size: 4
Current number of classes: 14
This conjugacy class has size: 2
Current number of classes: 15
Thi

This conjugacy class has size: 16
Current number of classes: 143
This conjugacy class has size: 32
Current number of classes: 144
This conjugacy class has size: 16
Current number of classes: 145
This conjugacy class has size: 32
Current number of classes: 146
This conjugacy class has size: 32
Current number of classes: 147
This conjugacy class has size: 32
Current number of classes: 148
This conjugacy class has size: 32
Current number of classes: 149
This conjugacy class has size: 4
Current number of classes: 150
This conjugacy class has size: 16
Current number of classes: 151
This conjugacy class has size: 8
Current number of classes: 152
This conjugacy class has size: 4
Current number of classes: 153
This conjugacy class has size: 8
Current number of classes: 154
This conjugacy class has size: 16
Current number of classes: 155
This conjugacy class has size: 8
Current number of classes: 156
This conjugacy class has size: 32
Current number of classes: 157
This conjugacy class has size: